<a href="https://colab.research.google.com/github/rezzix/python_ai_problems/blob/master/Titanic_5_models_average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Imports and data reading to dataframes

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("http://rezzix.mederp.net/data/titanic/train.csv")
df_tst = pd.read_csv("http://rezzix.mederp.net/data/titanic/test.csv")
print (df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Info, description, nulls ...

In [ ]:
#df.shape
df.describe()

In [ ]:
df.describe(include=['object', 'bool'])

In [ ]:
df.info()

In [ ]:
df_tst.info()

In [ ]:
df_agenotnull = df_tst.loc[df_tst['Age'].notnull()]

plt.scatter(df_agenotnull.Age, df_agenotnull.Pclass)
df_agenotnull

## 3. Some exploratory data analysis

In [ ]:
sns.boxplot(x=df['Survived'], y=df['Age'])

In [ ]:
sns.boxplot(x=df['Survived'], y=df["Fare"])

In [ ]:
"""
#df.describe(include=['object', 'bool'])
print(df['Embarked'].value_counts())

#the survival is correlated to the embarked feature let's classify it to make an ordinal feature 
print('S', df[['Embarked','Pclass','Survived']].loc[df['Embarked']=='S'].Survived.sum()/644)
print('C', df[['Embarked','Pclass','Survived']].loc[df['Embarked']=='C'].Survived.sum()/168)
print('Q', df[['Embarked','Pclass','Survived']].loc[df['Embarked']=='Q'].Survived.sum()/77)

embarked_mapper = {'S':1,'Q':2,'C':3}
df['Embarked'] = df['Embarked'].map(embarked_mapper)"""

## 4. Useful trasformations and feature engineering

In [ ]:
gender_mapper = {"male":0,"female":1}
df["Sex"] = df["Sex"].map(gender_mapper)
df_tst["Sex"] = df_tst["Sex"].map(gender_mapper)

In [ ]:
#df.set_index('PassengerId', inplace=True)
#df_tst.set_index('PassengerId', inplace=True)

In [ ]:
df['Cabin']=df['Cabin'].fillna('X')
df_tst['Cabin']=df_tst['Cabin'].fillna('X')

df['Cabin_pref']=df['Cabin'].str.slice(0,1)
df_tst['Cabin_pref']=df_tst['Cabin'].str.slice(0,1)

df['Embarked']=df['Embarked'].fillna('X')
df_tst['Embarked']=df_tst['Embarked'].fillna('X')

df['Age'].fillna(df['Age'].mean(), inplace=True)
df_tst['Age'].fillna(df_tst['Age'].mean(), inplace=True)

df['Fare'].fillna(df['Fare'].mean(), inplace=True)
df_tst['Fare'].fillna(df_tst['Fare'].mean(), inplace=True)

In [ ]:
df_cabsurv = df[['Cabin_pref','Survived']]
df['cabin_score'] = 0
cabin_mapper = {}

for pref in df['Cabin_pref'].unique():
    cabin_mapper.update({pref : df_cabsurv.loc[df['Cabin_pref'] == pref].Survived.sum()/df_cabsurv.loc[df['Cabin_pref'] == pref].size})

#the survival may be correlated to the Cabin_score feature let's classify it to make an ordinal feature 
df['cabin_score'] = df['Cabin_pref'].map(cabin_mapper) 
df_tst['cabin_score'] = df_tst['Cabin_pref'].map(cabin_mapper)    


df_embarksurv = df[['Embarked','Survived']]
df['embarked_score'] = 0
embarked_mapper = {}

for emb in df['Embarked'].unique():
    embarked_mapper.update({emb: df_embarksurv.loc[df['Embarked'] == emb].Survived.sum() / df_embarksurv.loc[df['Embarked'] == emb].size})

#the survival may be correlated to the embarked_score feature let's classify it to make an ordinal feature 
df['embarked_score'] = df['Embarked'].map(embarked_mapper)
df_tst['embarked_score'] = df_tst['Embarked'].map(embarked_mapper)

df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)
df_tst['Title'] = df_tst['Name'].str.extract('([A-Za-z]+)\.', expand=True)
df_titlesurv = df[['Title','Survived']]
df['title_score'] = 0
title_mapper = {}

for title in df['Title'].unique():
    title_mapper.update({title : df_titlesurv.loc[df['Title'] == title].Survived.sum()/df_titlesurv.loc[df['Title'] == title].size})
    
df['title_score'] = df['Title'].map(title_mapper)
df_tst['title_score'] = df_tst['Title'].map(title_mapper)

In [ ]:
df[['Pclass','Sex','Age','SibSp','Parch','Fare','cabin_score','embarked_score','title_score']]

df_tst['title_score'].fillna(df_tst['title_score'].mean(), inplace=True)

In [ ]:
plt.subplots(figsize=(10,8))
sns.heatmap(df[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','cabin_score','embarked_score','title_score']].corr(),annot=True,cmap='Blues_r')
plt.title("Correlation Among Variables", fontsize = 20);

## 5. Scaling and train test splitting

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_worked = scaler.fit_transform(df[['Pclass','Sex','Age','SibSp','Parch','Fare','cabin_score','embarked_score','title_score']])
df_tst_worked = scaler.transform(df_tst[['Pclass','Sex','Age','SibSp','Parch','Fare','cabin_score','embarked_score','title_score']])

df_worked

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_worked, df['Survived'].values, test_size=0.20, random_state=40)

## 6. Modeling and evaluating

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

logReg = LogisticRegression()
logReg.fit(X_train,y_train)

logReg.score(X_test, y_test)

In [ ]:
logReg_predict = logReg.predict(df_tst_worked)

In [ ]:
ids = df_tst['PassengerId']
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': logReg_predict })
output.to_csv('submission_logreg.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Random_forest = RandomForestClassifier(n_estimators=17)
Random_forest.fit(X_train,y_train)

randomForest_predict = Random_forest.predict(df_tst_worked)
randomForest_score = Random_forest.score(X_test,y_test)
# print("Random Forest Prediction :",RF_predict)
print("Random Forest Score :",randomForest_score)

In [ ]:
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': randomForest_predict })
output.to_csv('submission_randforst.csv', index=False)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN_model = KNeighborsClassifier(n_neighbors=7)
KNN_model.fit(X_train, y_train)

KNN_predict = KNN_model.predict(df_tst_worked)
print("K Neighbours :",KNN_model.score(X_test,y_test))

output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': KNN_predict })
output.to_csv('submission_knn.csv', index=False)

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier()
dtree_model = dtree_model.fit(X_train, y_train)

tree_predict = dtree_model.predict(df_tst_worked)
print("Decision Tree :",dtree_model.score(X_test,y_test))

'''
data = tree.export_graphviz(dtree, out_file=None, feature_names=['Pclass','Sex','Age','SibSp','Parch','Fare','cabin_score','embarked_score'])
graph = pydotplus.graph_from_dot_data(data)
graph.write_png('mydecisiontree.png')

img=pltimg.imread('mydecisiontree.png')
imgplot = plt.imshow(img)
plt.show()'''

In [ ]:
from sklearn.svm import SVC

SVC_model = SVC(probability=True)
SVC_model.fit(X_train,y_train)
SVC_predict = SVC_model.predict(df_tst_worked)
SVC_score = SVC_model.score(X_test,y_test)
#print("Support Vector Classifier Prediction :",SVC_predict)
print("Support Vector Classifier Score :",SVC_score)

NameError: ignored

In [ ]:
avg_predict = np.rint((randomForest_predict + KNN_predict + logReg_predict+SVC_predict+tree_predict)/5)

output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': avg_predict })
output.to_csv('submission_avg.csv', index=False)

NameError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(KNN_model.predict(X_test), y_test, labels=[1,0])

NameError: ignored